In [1]:
import matplotlib.pyplot as plt

from turtle import forward
import numpy as np
import pandas as pd
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

from torch.utils.data import Dataset, DataLoader

import time
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import time
from scipy.fft import fft

from pathlib import Path
import torchaudio
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms  

import torch.nn.functional as F





from sklearn.metrics import accuracy_score, confusion_matrix


def acc_m(test_dataset, model):
    device = next(model.parameters()).device
    y_test = []
    y_test_hat = []
    model.eval()
    
    for inputs, labels in test_dataset:
        # Add batch dimension like in training
        inputs = inputs.unsqueeze(0)  # [freq, time] -> [1, freq, time]
        
        with torch.no_grad():
            outputs = model(inputs.to(device))
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
        
        y_test.append(labels)  # labels is already an int
        y_test_hat.extend(preds)

    acc = accuracy_score(y_test, y_test_hat)
    print(f'Accuracy: {acc*100:.2f} %')
    cm = confusion_matrix(y_test, y_test_hat)
    print(cm)

In [22]:


class AudioFolder(Dataset):
    def __init__(self, root, transform=None, target_transform=None,
                 extensions=(".wav", ".flac", ".mp3")):
        self.root = Path(root)
        self.transform = transform
        self.target_transform = target_transform
        self.extensions = extensions

        # crawl sub-dirs and record (path, class_idx)
        self.classes = sorted({p.parent.name for p in self.root.rglob("*") if p.suffix in extensions})
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
        self.samples = [(p, self.class_to_idx[p.parent.name])
                        for p in self.root.rglob("*") if p.suffix in extensions]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        waveform, sr = torchaudio.load(path, backend="ffmpeg")

        if self.transform:
            waveform = self.transform(waveform)
        if self.target_transform:
            label = self.target_transform(label)

        return waveform[0], label


transform_wav = lambda x: torchaudio.functional.spectrogram(
        x,
        pad=0,
        win_length = 255,
        window=torch.hann_window(255),  # Hann window
        n_fft=255,
        normalized=False,
        center=False ,
        hop_length=128,                  # overlap = n_fft - hop_length
        power=None                       # None -> keep complex, 2.0 -> magnitude^2
    )
dataset_wav = AudioFolder('../data/hb/', transform_wav)
dataset_wav_train, dataset_wav_test = train_test_split(dataset_wav, train_size=0.9)


class AudioClassifier(nn.Module):
    def __init__(self, n_classes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        def build_Seq():
            return nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            # nn.BatchNorm2d(16),
            nn.Conv2d(32, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            )
        self.real = build_Seq()

        self.imag = build_Seq()

        in_size = 24000 * 2
        self.fc = nn.Sequential(
            nn.Linear(in_size, in_size // 8),
            nn.Linear(in_size // 8, in_size // 32),
            nn.Linear(in_size // 32, n_classes)
        )

    def forward(self, x):
        real_input = x.real.unsqueeze(1)
        img_input = x.imag.unsqueeze(1)

        out_l = self.real(real_input)   
        out_r = self.imag(img_input)        

        out_l = F.adaptive_avg_pool2d(out_l, (30, 100))  # Fixed reasonable size
        out_r = F.adaptive_avg_pool2d(out_r, (30, 100))  # [batch, 8, 30, 100]

        out_l = out_l.flatten(1)  # [batch, 24000]
        out_r = out_r.flatten(1)  # [batch, 24000]
        out = torch.cat([out_l, out_r], dim=1)  # [batch, 2*channels*freq*time]
        return self.fc(out)
    

class AudioClassifier2(nn.Module):
    def __init__(self, n_classes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        def build_Seq():
            return nn.Sequential(
                nn.Conv2d(1, 6, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2,2),
                nn.BatchNorm2d(6),
                nn.Conv2d(6, 16, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2,2),
                nn.BatchNorm2d(16),
                nn.AdaptiveAvgPool2d((16, 32)),  # [batch, 8, 30, 100]
                nn.Flatten(1)
            )
        self.real = build_Seq()

        self.imag = build_Seq()

        in_size = 16384
        self.fc = nn.Sequential(
            nn.Linear(in_size, 128),
            nn.Linear(128, 64),
            nn.Linear(64, n_classes)
        )

    def forward(self, x):
        # print(x.real.shape)
        real_input = x.real.unsqueeze(1)
        img_input = x.imag.unsqueeze(1)
        # real_input = x.real
        # img_input = x.imag     

        out_l = self.real(real_input)   
        out_r = self.imag(img_input)        

        out = torch.cat([out_l, out_r], dim=1)  # [batch, 2*channels*freq*time]
        # print(out.shape)
        return self.fc(out)
    
    
ac = AudioClassifier2(len(dataset_wav.classes))
res_t = ac(dataset_wav_test[0][0].unsqueeze(0))
res_t.shape


torch.Size([1, 6])

In [59]:
import random
data_path = '../data/hb/'

for i in range(10):
    unl = 'unlabelled'
    unlab = Path(data_path,unl)
    random_unlabel = random.choice(list(unlab.glob('*')))
    waveform, sr = torchaudio.load(random_unlabel, backend="ffmpeg")
    transformed = fft(waveform.reshape(-1).numpy())
    print(transformed.shape)
# STFT: windowing + overlap + FFT all in one
    spec = torchaudio.functional.spectrogram(
        waveform,
        pad=0,
        win_length = 255,
        window=torch.hann_window(255),  # Hann window
        n_fft=255,
        normalized=False,
        center=False ,
        hop_length=128,                  # overlap = n_fft - hop_length
        power=None                       # None -> keep complex, 2.0 -> magnitude^2
    )
    print(spec.shape)    

(41625,)
torch.Size([1, 128, 324])
(48057,)
torch.Size([1, 128, 374])
(49400,)
torch.Size([1, 128, 384])
(47952,)
torch.Size([1, 128, 373])
(47952,)
torch.Size([1, 128, 373])
(60968,)
torch.Size([1, 128, 475])
(48057,)
torch.Size([1, 128, 374])
(50145,)
torch.Size([1, 128, 390])
(47092,)
torch.Size([1, 128, 366])
(47092,)
torch.Size([1, 128, 366])


In [141]:
sm = dataset_wav_test.pop

In [145]:
# sm = sm()
sm[0].shape

torch.Size([128, 428])

In [ ]:

train_dataset = dataset_wav_train
test_dataset = dataset_wav_test


loss = nn.CrossEntropyLoss()
device = 'cuda'
ac = ac.to(device)
optimiser = torch.optim.Adam(ac.parameters(), lr=0.0001)

NUM_EPOCHS = 20
losses = []

for epoch in range(NUM_EPOCHS):
    for i, (inputs, labels) in enumerate(train_dataset):
        optimiser.zero_grad()
        
        # Add batch dimension manually
        inputs = inputs.unsqueeze(0)  # [1, freq, time]
        labels = torch.tensor([labels])  # [1]
        
        y_pred = ac(inputs.to(device))
        loss_v = loss(y_pred, labels.to(device))
        
        loss_v.backward()
        losses.append(loss_v.item())
        optimiser.step()
        
        if i % 50 == 0:
            print(f'Epoch {epoch}/{NUM_EPOCHS}, Sample {i}, Loss: {loss_v.item():.4f}')
    
    print(f'Completed Epoch {epoch+1}')
    acc_m(test_dataset, ac)

Epoch 0/20, Sample 0, Loss: 1.7950
Epoch 0/20, Sample 50, Loss: 0.8155
Epoch 0/20, Sample 100, Loss: 1.5056
Epoch 0/20, Sample 150, Loss: 1.7920
Completed Epoch 1
Accuracy: 16.67 %
[[0 0 0 2 0]
 [0 0 0 1 0]
 [0 0 0 7 0]
 [0 0 0 3 0]
 [0 0 0 5 0]]
Epoch 1/20, Sample 0, Loss: 1.6342
Epoch 1/20, Sample 50, Loss: 1.6106
Epoch 1/20, Sample 100, Loss: 1.6471
Epoch 1/20, Sample 150, Loss: 1.8081
Completed Epoch 2
Accuracy: 16.67 %
[[0 0 0 2 0]
 [0 0 0 1 0]
 [0 0 0 7 0]
 [0 0 0 3 0]
 [0 0 0 5 0]]
Epoch 2/20, Sample 0, Loss: 1.5690
Epoch 2/20, Sample 50, Loss: 1.4755
Epoch 2/20, Sample 100, Loss: 1.6046
Epoch 2/20, Sample 150, Loss: 1.7318
Completed Epoch 3
Accuracy: 16.67 %
[[0 0 0 2 0]
 [0 0 0 1 0]
 [0 0 0 7 0]
 [0 0 0 3 0]
 [0 0 0 5 0]]
Epoch 3/20, Sample 0, Loss: 1.5456
Epoch 3/20, Sample 50, Loss: 1.3942
Epoch 3/20, Sample 100, Loss: 1.2745
Epoch 3/20, Sample 150, Loss: 1.9595
Completed Epoch 4
Accuracy: 27.78 %
[[0 0 0 0 2]
 [0 0 0 0 1]
 [0 0 0 0 7]
 [0 0 0 0 3]
 [0 0 0 0 5]]
Epoch 4/20, 

In [ ]:

batch_size = 1 # should've convert that into spectogram
trainloader = DataLoader(dataset_wav_train, batch_size=batch_size, shuffle=True, num_workers=8)
testloader = DataLoader(dataset_wav_test, batch_size=batch_size, shuffle=False,num_workers=8)


from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

loss = nn.CrossEntropyLoss()
device = 'cuda'
ac = ac.to(device)
optimiser = torch.optim.Adam(ac.parameters())

NUM_EPOCHS = 100
losses = []

for epoch in range(NUM_EPOCHS):
    for i, (inputs, labels) in enumerate(trainloader, 0):
        optimiser.zero_grad()

        
        y_pred = ac(inputs.to(device))
        loss_v = loss(y_pred, labels.to(device))
        
        loss_v.backward()
        losses.append(loss_v.item())
        optimiser.step()
        
        if i % 50 == 0:
            print(f'Epoch {epoch}/{NUM_EPOCHS}, Sample {i}, Loss: {loss_v.item():.4f}')
    
    print(f'Completed Epoch {epoch+1}')
    ac.eval()
    with torch.no_grad() :            
        y_real = []
        y_preds = []
        for i, (inputs, labels) in enumerate(testloader, 0):
            y_real.append(labels)
            y_preds.append(torch.argmax(ac(inputs.to(device)), dim=1).cpu().numpy())
        y_real = np.asarray(y_real).flatten()
        y_preds = np.asarray(y_preds).flatten()
        cm = confusion_matrix(y_real, y_preds)

        print("\nPer-class accuracy:")
        for i, cls in enumerate(dataset_wav.classes):
            class_acc = cm[i,i] / cm[i].sum() if cm[i].sum() > 0 else 0
            print(f"{cls}: {class_acc*100:.1f}% ({cm[i,i]}/{cm[i].sum()})")

        # Total accuracy
        total_acc = cm.diagonal().sum() / cm.sum()
        print(f"Total Accuracy: {total_acc*100:.2f}%")            
    ac.train()

Epoch 0/100, Sample 0, Loss: 1.7051
Epoch 0/100, Sample 50, Loss: 1.9741
Epoch 0/100, Sample 100, Loss: 0.9772
Epoch 0/100, Sample 150, Loss: 1.8452
Completed Epoch 1

Per-class accuracy:
artifact: 100.0% (1/1)
extrahls: 0.0% (0/1)
extrastole: 0.0% (0/9)
murmur: 50.0% (1/2)
normal: 0.0% (0/5)
unlabelled: 0.0% (0/0)
Total Accuracy: 11.11%
Epoch 1/100, Sample 0, Loss: 0.0001
Epoch 1/100, Sample 50, Loss: 0.0096
Epoch 1/100, Sample 100, Loss: 0.0000
Epoch 1/100, Sample 150, Loss: 0.0000
Completed Epoch 2

Per-class accuracy:
artifact: 100.0% (1/1)
extrahls: 0.0% (0/1)
extrastole: 11.1% (1/9)
murmur: 0.0% (0/2)
normal: 0.0% (0/5)
unlabelled: 0.0% (0/0)
Total Accuracy: 11.11%
Epoch 2/100, Sample 0, Loss: 0.0036
Epoch 2/100, Sample 50, Loss: 0.0001
Epoch 2/100, Sample 100, Loss: 0.0001
Epoch 2/100, Sample 150, Loss: 0.0011
Completed Epoch 3

Per-class accuracy:
artifact: 100.0% (1/1)
extrahls: 0.0% (0/1)
extrastole: 11.1% (1/9)
murmur: 0.0% (0/2)
normal: 0.0% (0/5)
unlabelled: 0.0% (0/0)
Tot

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f5c295cadc0>
Traceback (most recent call last):
  File "/home/not7/src/Vladimir-Ershov.github.io/projects/MLRecap/.pixi/envs/pytorch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/home/not7/src/Vladimir-Ershov.github.io/projects/MLRecap/.pixi/envs/pytorch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
  File "/home/not7/src/Vladimir-Ershov.github.io/projects/MLRecap/.pixi/envs/pytorch/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Epoch 5/100, Sample 100, Loss: 0.0008


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f5c295cadc0>
Traceback (most recent call last):
  File "/home/not7/src/Vladimir-Ershov.github.io/projects/MLRecap/.pixi/envs/pytorch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/home/not7/src/Vladimir-Ershov.github.io/projects/MLRecap/.pixi/envs/pytorch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
  File "/home/not7/src/Vladimir-Ershov.github.io/projects/MLRecap/.pixi/envs/pytorch/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Epoch 5/100, Sample 150, Loss: 0.0007
Completed Epoch 6

Per-class accuracy:
artifact: 100.0% (1/1)
extrahls: 0.0% (0/1)
extrastole: 11.1% (1/9)
murmur: 0.0% (0/2)
normal: 0.0% (0/5)
unlabelled: 0.0% (0/0)
Total Accuracy: 11.11%
Epoch 6/100, Sample 0, Loss: 0.0001
Epoch 6/100, Sample 50, Loss: 0.0000
Epoch 6/100, Sample 100, Loss: 0.0000
Epoch 6/100, Sample 150, Loss: 0.0001
Completed Epoch 7

Per-class accuracy:
artifact: 100.0% (1/1)
extrahls: 0.0% (0/1)
extrastole: 11.1% (1/9)
murmur: 0.0% (0/2)
normal: 0.0% (0/5)
unlabelled: 0.0% (0/0)
Total Accuracy: 11.11%
Epoch 7/100, Sample 0, Loss: 0.0000
Epoch 7/100, Sample 50, Loss: 0.0010
Epoch 7/100, Sample 100, Loss: 0.0002
Epoch 7/100, Sample 150, Loss: 0.0003
Completed Epoch 8

Per-class accuracy:
artifact: 100.0% (1/1)
extrahls: 0.0% (0/1)
extrastole: 11.1% (1/9)
murmur: 0.0% (0/2)
normal: 0.0% (0/5)
unlabelled: 0.0% (0/0)
Total Accuracy: 11.11%
Epoch 8/100, Sample 0, Loss: 0.0000
Epoch 8/100, Sample 50, Loss: 0.0002
Epoch 8/100, Sampl

KeyboardInterrupt: 

In [25]:
def print_class_distribution(data_subset, original_dataset, name="Dataset"):
    class_counts = {}
    for _, label in data_subset:
        class_counts[label] = class_counts.get(label, 0) + 1
    
    print(f"\n{name} - Class Distribution:")
    print(f"{'Class':<15} {'Count':<10} {'Percentage':<10}")
    print("-" * 35)
    total = len(data_subset)
    for class_name in original_dataset.classes:
        class_idx = original_dataset.class_to_idx[class_name]
        count = class_counts.get(class_idx, 0)
        pct = (count / total * 100) if total > 0 else 0
        print(f"{class_name:<15} {count:<10} {pct:>6.1f}%")
    print(f"{'Total':<15} {total:<10}")

# Use original dataset for class info
print_class_distribution(dataset_wav_train, dataset_wav, "Training Set")
print_class_distribution(dataset_wav_test, dataset_wav, "Test Set")


Training Set - Class Distribution:
Class           Count      Percentage
-----------------------------------
artifact        39           24.1%
extrahls        18           11.1%
extrastole      37           22.8%
murmur          32           19.8%
normal          26           16.0%
unlabelled      10            6.2%
Total           162       

Test Set - Class Distribution:
Class           Count      Percentage
-----------------------------------
artifact        1             5.6%
extrahls        1             5.6%
extrastole      9            50.0%
murmur          2            11.1%
normal          5            27.8%
unlabelled      0             0.0%
Total           18        


In [ ]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset

# Get all labels for stratification
all_labels = [label for _, label in dataset_wav]

# Cross-validation loop
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_idx, test_idx) in enumerate(skf.split(range(len(dataset_wav)), all_labels)):
    print(f"\n{'='*50}")
    print(f"FOLD {fold + 1}/5")
    print(f"{'='*50}")
    
    # Create subsets for this fold
    train_subset = Subset(dataset_wav, train_idx)
    test_subset = Subset(dataset_wav, test_idx)
    
    # Create dataloaders
    trainloader = DataLoader(train_subset, batch_size=1, shuffle=True, num_workers=8)
    testloader = DataLoader(test_subset, batch_size=1, shuffle=False, num_workers=8)
    
    # Fresh model for each fold
    ac = AudioClassifier2(len(dataset_wav.classes)).to('cuda')
    optimiser = torch.optim.Adam(ac.parameters(), lr=0.0001, weight_decay=1e-4)
    loss = nn.CrossEntropyLoss()
    
    # Training loop
    for epoch in range(20):  # Fewer epochs per fold
        ac.train()
        for i, (inputs, labels) in enumerate(trainloader):
            optimiser.zero_grad()
            y_pred = ac(inputs.to('cuda'))
            loss_v = loss(y_pred, labels.to('cuda'))
            loss_v.backward()
            optimiser.step()
        
        if epoch % 5 == 0:
            print(f"Fold {fold+1}, Epoch {epoch}, Loss: {loss_v.item():.4f}")
    
    # Evaluate this fold
    ac.eval()
    y_real, y_preds = [], []
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = ac(inputs.to('cuda'))
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_real.extend(labels.numpy())
            y_preds.extend(preds)
    
    acc = (np.array(y_real) == np.array(y_preds)).mean()
    fold_accuracies.append(acc)
    print(f"Fold {fold+1} Accuracy: {acc*100:.2f}%")

print(f"\nAverage Accuracy: {np.mean(fold_accuracies)*100:.2f}% (+/- {np.std(fold_accuracies)*100:.2f}%)")


FOLD 1/5
Fold 1, Epoch 0, Loss: 2.2299
Fold 1, Epoch 5, Loss: 0.0026
Fold 1, Epoch 10, Loss: 0.0016
Fold 1, Epoch 15, Loss: 0.0006
Fold 1 Accuracy: 22.22%

FOLD 2/5
Fold 2, Epoch 0, Loss: 1.5439
Fold 2, Epoch 5, Loss: 0.0060
Fold 2, Epoch 10, Loss: 0.0013
Fold 2, Epoch 15, Loss: 0.0006
Fold 2 Accuracy: 19.44%

FOLD 3/5
Fold 3, Epoch 0, Loss: 1.9473
Fold 3, Epoch 5, Loss: 0.0087
Fold 3, Epoch 10, Loss: 0.0012
Fold 3, Epoch 15, Loss: 0.0006
Fold 3 Accuracy: 11.11%

FOLD 4/5
Fold 4, Epoch 0, Loss: 1.4087
Fold 4, Epoch 5, Loss: 0.0045
Fold 4, Epoch 10, Loss: 0.0023
Fold 4, Epoch 15, Loss: 0.0005
Fold 4 Accuracy: 25.00%

FOLD 5/5
Fold 5, Epoch 0, Loss: 1.8590
Fold 5, Epoch 5, Loss: 0.0048
Fold 5, Epoch 10, Loss: 0.0015
Fold 5, Epoch 15, Loss: 0.0008
Fold 5 Accuracy: 22.22%

Average Accuracy: 20.00% (+/- 4.78%)


In [30]:
from collections import Counter


Counter(y_real)

Counter({2: 10, 0: 8, 4: 7, 3: 6, 1: 3, 5: 2})